In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import sys
import time
import gc
import itertools
import tqdm
import time
import copy

import scipy.stats as ss
from numba import jit

if os.name=='nt':
    try:
        mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
        os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
    except:
        pass
    
import xgboost as xgb
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

%matplotlib inline

### Helper Function

In [2]:
def lag_features(df, lags, merge_cols, shift_cols, fillna_value=None):
    '''create lag features of col'''
    cols = copy.copy(merge_cols)
    cols.extend(shift_cols)
    tmp = df.loc[:, cols]
    for i in lags:
        shifted = tmp.copy()
        shifted_cols = shifted.columns.tolist()
        shifted_cols = [c+'_lag_'+str(i) if c in shift_cols else c 
                        for c in shifted_cols]
        shifted.columns = shifted_cols
        shifted['month'] += i
        shifted.drop_duplicates(inplace=True)
        df = pd.merge(df, shifted, on=merge_cols, how='left')
    if fillna_value is not None:
        df.fillna(fillna_value, inplace=True)
    return df    

In [3]:
def clip_rmse(preds, dtrain):
    y_test = np.array(dtrain.get_label())
    preds = np.array(preds)
    y_test = np.maximum(np.minimum(y_test, 20), 0)
    preds = np.maximum(np.minimum(preds, 20), 0)
    #preds = np.array(list(map(lambda x: min(20, max(x, 0)), list(preds))))
    #y_test = np.array(list(map(lambda x: min(20, max(x, 0)), list(y_test))))
    rmse = np.sqrt(mean_squared_error(preds,y_test))
    return 'clip-rmse', rmse

In [4]:
def cv(x_train, y_train, param, n_repetition, n_split, n_tree, verbose, random_state):
    '''Repeated CV'''
    
    cv_results = {}
    clf = {}
    running_time = {}

    np.random.seed(random_state)

    for m in range(n_repetition):
        # Train and valuation sets split
        skf = StratifiedKFold(n_splits=n_split, random_state=np.random.randint(10**6), shuffle=True)

        for n, (train_index, val_index) in enumerate(skf.split(x_train, y_train)):

            start_time = time.time()

            # Construct DMatrix
            dtrain = xgb.DMatrix(x_train.iloc[train_index], label=y_train.iloc[train_index])
            dval = xgb.DMatrix(x_train.iloc[val_index], label=y_train.iloc[val_index])

            # Placeholder for evals_results
            cv_results[m, n] = {}

            param['seed'] = np.random.randint(10**6)
            clf[m, n] = xgb.train(param, dtrain,num_boost_round=n_tree, 
                                  evals=[(dtrain, 'train'), (dval, 'val')], 
                                  feval=clip_rmse, maximize=False, early_stopping_rounds=None, 
                                  evals_result=cv_results[m, n], verbose_eval=verbose)

            running_time[m, n] = time.time() - start_time

            print('Repeat {}, split {}, val score = {:.3f}, running time = {:.3f} min.'.format(m, n, 
                cv_results[m, n]['val']['clip-rmse'][-1], running_time[m, n]/60))

    cv_results_final = {}
    for m in range(n_repetition):
        for n in range(n_split):
            cv_results_final['train', m, n] = cv_results[m, n]['train']['clip-rmse']
            cv_results_final['val', m, n] = cv_results[m, n]['val']['clip-rmse']

    df = pd.DataFrame(cv_results_final)
    df.index.name = 'iteration'
    df.columns.names = ['dataset', 'repetition', 'cv_split']

    print('Val mean = {:.3f}, std = {:.3f}'.format(df['val'].iloc[-1].mean(), df['val'].iloc[-1].std()))
    
    return df, clf, running_time

In [5]:
def feature_combination(x, feature_list, function_dict, column_name, merge=False):
    '''Combination of new features'''
    tmp = x.groupby(feature_list).agg(function_dict)
    tmp.columns = column_name
    if merge:
        x = x.merge(tmp, on=feature_list, how='left')
        return x, tmp
    else:
        return tmp

### Load and Preprocess Data

In [6]:
# Load data
train = pd.read_csv('all/sales_train.csv.gz')
test = pd.read_csv('all/test.csv.gz')
shop = pd.read_csv('all/shops-translated.csv')
item = pd.read_csv('all/item_category.csv')

test.set_index('ID', inplace=True)
item.drop(['item_name_translated'], axis=1, inplace=True)
shop.drop(['Name'], axis=1, inplace=True)

le = LabelEncoder()
item['item_cat1'] = le.fit_transform(item['item_cat1'].astype(str))
item['item_cat2'] = le.fit_transform(item['item_cat2'].astype(str))
shop['City'] = le.fit_transform(shop['City'])
shop['Type'] = le.fit_transform(shop['Type'])

Remove outliers

In [7]:
train = train[train.item_price<100000]
train = train[train.item_cnt_day<1001]
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

Fix shop names and ids

In [8]:
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

Monthly sales for all the samples

In [9]:
x = pd.DataFrame(train.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum())

x.reset_index(inplace=True)

Training set only contains sold samples, need to extend to all samples.

There are two ways of extending:
1. overall product between elements in (month, shop_id, item_id)
2. in each month, the product between elements in (shop_id, item_id)

The first one increases the number of rows by 23.5 times, the second one 6 times

In [10]:
shop_count = {}
item_count = {}
product_count = {}
sample_count = {}
ratio = {}

for n in x.date_block_num.unique():
    shop_count[n] = len(x.loc[x.date_block_num==n, 'shop_id'].unique())
    item_count[n] = len(x.loc[x.date_block_num==n, 'item_id'].unique())
    sample_count[n] = len(x.loc[x.date_block_num==n, :])
    product_count[n] = shop_count[n]*item_count[n]
    ratio[n] = product_count[n]/sample_count[n]
#     print('product count is {}, sample count is {}, ratio is {:.3f}'.format(product_count[n], 
#                                                                             sample_count[n], 
#                                                                             product_count[n]/sample_count[n]))
    
print('overall product count is {}, sample count is {}, ratio is {:.3f}'.format(
    len(x.shop_id.unique())*len(x.item_id.unique())*34, 
    x.shape[0], 34*len(x.shop_id.unique())*len(x.item_id.unique())/x.shape[0]))
print('monthly product count sum is {}, monthly sample count is {}, ratio is {:.3f}'.format(
    sum(product_count.values()), sum(sample_count.values()), sum(product_count.values())/sum(sample_count.values())))

overall product count is 42260028, sample count is 1609123, ratio is 26.263
monthly product count sum is 10913804, monthly sample count is 1609123, ratio is 6.782


Try the first extending method

It requires huge RAM

In [11]:
if False:
    month = np.arange(0, 34)
    shop_list = train.shop_id.unique().tolist()
    item_list = train.item_id.unique().tolist()
    n_rows = len(month)*len(shop_list)*len(item_list)

    idx = pd.MultiIndex.from_product([month, shop_list, item_list], names=['date_block_num', 'shop_id', 'item_id'])

    x2 = pd.DataFrame(np.zeros((n_rows,2)), index=idx)
    x2.reset_index(inplace=True, drop=False)
    x2.drop([0, 1], axis=1, inplace=True)

    x = x2.merge(x, on=['date_block_num', 'shop_id', 'item_id'], how='outer').fillna(0.0)
    test['date_block_num'] = 34
    x = pd.concat((x, test), sort=False).fillna(0.0)

    del x2
    gc.collect()

Try the second method

In [12]:
tmp = []
for n in range(34):
    shop_list = x.loc[x.date_block_num==n, 'shop_id'].unique()
    item_list = x.loc[x.date_block_num==n, 'item_id'].unique()
    idx = pd.MultiIndex.from_product([[n], shop_list, item_list], names=['date_block_num', 'shop_id', 'item_id'])
    df_tmp = pd.DataFrame(np.zeros((len(idx),2)), index=idx)
    tmp.append(df_tmp)
tmp = pd.concat(tmp, sort=False)
tmp.reset_index(inplace=True, drop=False)
tmp.drop([0, 1], axis=1, inplace=True)
x = tmp.merge(x, on=['date_block_num', 'shop_id', 'item_id'], how='outer').fillna(0.0)
test['date_block_num'] = 34
x = pd.concat((x, test), sort=False).fillna(0.0)

Add shop/item features

In [13]:
x = pd.merge(x, shop, on='shop_id', how='left')
x = pd.merge(x, item, on='item_id', how='left')

Change column names

In [14]:
x.columns = ['month', 'shop_id', 'item_id', 'sales_month', 
             'City', 'Type', 'item_cat1', 'item_cat2']

Distance to the last sold month for (month, shop, item) pair

In [15]:
tmp = x.pivot_table(index=['shop_id', 'item_id'], 
                    columns=['month'], 
                    values=['sales_month']).fillna(0.0)

tmp = tmp>0.0
for n in tqdm.tqdm_notebook(range(tmp.shape[1])):
    tmp.iloc[:, n] = tmp.iloc[:, n].map({False: -1, True: n})
tmp = tmp.shift(axis=1).fillna(-1.0).cummax(axis=1)
tmp = np.arange(0, 35)-tmp.replace(-1.0, -np.inf)
tmp.replace(np.inf, -1, inplace=True)
tmp = tmp.stack()
tmp.columns = ['shop_item_to_last_sold_month']
tmp.reset_index(inplace=True)
tmp = tmp[['month', 'shop_id', 'item_id', 'shop_item_to_last_sold_month']]

x = pd.merge(x, tmp, on=['month', 'shop_id', 'item_id'], how='left')

In [16]:
tmp = x.pivot_table(index=['item_id'], 
                    columns=['month'], 
                    values=['sales_month']).fillna(0.0)
tmp = tmp>0.0
for n in tqdm.tqdm_notebook(range(tmp.shape[1])):
    tmp.iloc[:, n] = tmp.iloc[:, n].map({False: -1, True: n})
tmp = tmp.shift(axis=1).fillna(-1.0).cummax(axis=1)
tmp = np.arange(0, 35)-tmp.replace(-1.0, -np.inf)
tmp.replace(np.inf, -1, inplace=True)
tmp = tmp.stack()
tmp.columns = ['item_to_last_sold_month']
tmp.reset_index(inplace=True)
tmp = tmp[['month', 'item_id', 'item_to_last_sold_month']]

x = pd.merge(x, tmp, on=['month', 'item_id'], how='left')

In [18]:
tmp = x.pivot_table(index=['shop_id', 'item_id'], 
                    columns=['month'], 
                    values=['sales_month'], 
                    aggfunc=np.sum).fillna(0.0)

a = np.nonzero(tmp.values)
a = np.array(a).T
a = pd.DataFrame(a)
a.columns = ['index', 'shop_item_to_first_sold_month']

a = pd.DataFrame(a.groupby('index')['shop_item_to_first_sold_month'].min())

b = tmp.copy()
b.reset_index(inplace=True, drop=False)
b.drop('sales_month', level=0, axis=1, inplace=True)
b.columns = ['shop_id', 'item_id']
b = b.join(a, how='left').fillna(-1.0)
x = x.merge(b, on=['shop_id', 'item_id'], how='left')

In [19]:
tmp = x.pivot_table(index=['item_id'], 
                    columns=['month'], 
                    values=['sales_month'], 
                    aggfunc=np.sum).fillna(0.0)


a = np.nonzero(tmp.values)
a = np.array(a).T
a = pd.DataFrame(a)
a.columns = ['index', 'item_to_first_sold_month']
a = pd.DataFrame(a.groupby('index')['item_to_first_sold_month'].min())

b = tmp.copy()
b.reset_index(inplace=True, drop=False)
b.drop('sales_month', level=0, axis=1, inplace=True)
b.columns = ['item_id']

b = b.join(a, how='left').fillna(-1.0)
x = x.merge(b, on=['item_id'], how='left')

In [20]:
x.sample(50)

,month,shop_id,item_id,sales_month,City,Type,item_cat1,item_cat2,shop_item_to_last_sold_month,item_to_last_sold_month,shop_item_to_first_sold_month,item_to_first_sold_month
693374,1,53,3187,0.0,21,4,9,42,-1.0,1.0,2.0,0.0
7784509,22,2,4607,0.0,0,4,12,16,-1.0,1.0,-1.0,0.0
2857152,7,35,17579,0.0,10,5,3,23,4.0,4.0,3.0,0.0
2322085,6,12,3556,0.0,25,6,9,58,4.0,1.0,2.0,0.0
9050547,26,7,3364,0.0,24,5,9,58,2.0,1.0,18.0,16.0
6819160,18,59,15886,0.0,27,1,10,28,4.0,1.0,13.0,11.0
3192747,8,30,9363,0.0,8,4,10,51,4.0,1.0,2.0,0.0
1900944,5,5,16028,0.0,22,5,10,12,1.0,1.0,3.0,3.0
10567487,32,34,13747,0.0,10,5,10,51,-1.0,1.0,-1.0,22.0
4748781,12,47,2129,0.0,18,2,12,16,1.0,1.0,11.0,9.0
